In [19]:
from pyspark.sql.functions import col

In [1]:
import pyspark
from pyspark.sql import SparkSession


spark = SparkSession.builder \
        .master("local[*]") \
        .appName('teste') \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/27 14:29:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/raw/green/*/*')
df_yellow = spark.read.parquet('data/raw/yellow/*/*')

In [3]:
df_green.createOrReplaceTempView('green')
df_yellow.createOrReplaceTempView('yellow')

In [4]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
GROUP BY
    1, 2
""")

In [5]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [6]:
df_green_revenue.show()

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
| 174|2019-12-31 21:00:00|              21.7|             1|
|  41|2019-12-31 23:00:00|            556.11|            36|
| 173|2020-01-01 02:00:00| 59.03999999999999|             3|
|  75|2020-01-01 04:00:00|127.15999999999998|            10|
|   7|2020-01-01 14:00:00| 357.8400000000001|            28|
|  36|2020-01-01 13:00:00|             18.09|             2|
|  41|2020-01-01 15:00:00| 487.9300000000001|            33|
|  25|2020-01-01 16:00:00|            262.92|            12|
| 179|2020-01-01 20:00:00|            139.97|             6|
|  62|2020-01-02 07:00:00|             75.78|             6|
| 181|2020-01-02 14:00:00|            179.72|            10|
|  83|2020-01-02 15:00:00|             53.89|             3|
|  92|2020-01-02 15:00:00|            160.36|             8|
| 189|2020-01-02 17:00:0

In [7]:
df_yellow_revenue.show()

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
|   1|2020-01-01 00:00:00|              95.3|             1|
|   1|2020-01-01 01:00:00|              94.8|             1|
|   1|2020-01-01 02:00:00|              40.3|             1|
|   1|2020-01-01 03:00:00|445.28000000000003|             4|
|   1|2020-01-01 04:00:00|              0.31|             1|
|   1|2020-01-01 05:00:00|              60.8|             1|
|   1|2020-01-01 06:00:00|            175.26|             2|
|   1|2020-01-01 07:00:00|             100.6|             2|
|   1|2020-01-01 08:00:00|              90.3|             1|
|   1|2020-01-01 09:00:00|266.65999999999997|             3|
|   1|2020-01-01 10:00:00|            213.36|             2|
|   1|2020-01-01 11:00:00|             239.5|             3|
|   1|2020-01-01 12:00:00|            315.21|             4|
|   1|2020-01-01 13:00:0

In [8]:
df_green_revenue.repartition(20).write.parquet('data/report/revenue/green',mode='overwrite')

23/03/27 14:29:53 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


23/03/27 14:29:54 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [9]:
df_yellow_revenue.repartition(20).write.parquet('data/report/revenue/yellow',mode='overwrite')

23/03/27 14:30:05 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


23/03/27 14:30:06 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [10]:
df_green_revenue_tmp = df_green_revenue \
                .withColumnRenamed('amount', 'green_amount') \
                .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
                .withColumnRenamed('amount', 'yellow_amount') \
                .withColumnRenamed('number_records', 'yellow_number_records')

In [11]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [12]:
df_join.show()

+-------------------+----+------------+--------------------+------------------+---------------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+------------------+---------------------+
|2020-01-01 01:00:00|   1|        null|                null|              94.8|                    1|
|2020-01-01 05:00:00|   1|        null|                null|              60.8|                    1|
|2020-01-01 10:00:00|   1|        null|                null|            213.36|                    2|
|2020-01-03 02:00:00|   1|      117.39|                   1|             100.1|                    1|
|2020-01-03 03:00:00|   1|        null|                null|            174.36|                    1|
|2020-01-03 11:00:00|   1|        null|                null|447.39000000000004|                    5|
|2020-01-04 12:00:00|   1|        null|                null|460.06000000000006|   

In [14]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

23/03/27 14:31:45 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [15]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv

--2023-03-27 14:37:45--  https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv
Resolvendo d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2123:4200:b:20a5:b140:21, 2600:9000:2123:4600:b:20a5:b140:21, 2600:9000:2123:ea00:b:20a5:b140:21, ...
Conectando-se a d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2123:4200:b:20a5:b140:21|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 12322 (12K) [text/csv]
Salvando em: “taxi+_zone_lookup.csv”

taxi+_zone_lookup.c 100%[===================>]  12,03K  --.-KB/s    em 0s      

2023-03-27 14:37:46 (166 MB/s) - “taxi+_zone_lookup.csv” salvo [12322/12322]



In [17]:
df_zones = spark.read.option('header','true').csv('zones/')

In [20]:
df_zones = df_zones.withColumn("LocationID", col("LocationID").cast("int"))

In [21]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [22]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [23]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [25]:
df_result.drop('LocationID').show()

+-------------------+----+------------+--------------------+------------------+---------------------+-------+--------------+------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|Borough|          Zone|service_zone|
+-------------------+----+------------+--------------------+------------------+---------------------+-------+--------------+------------+
|2020-01-01 01:00:00|   1|        null|                null|              94.8|                    1|    EWR|Newark Airport|         EWR|
|2020-01-01 05:00:00|   1|        null|                null|              60.8|                    1|    EWR|Newark Airport|         EWR|
|2020-01-01 10:00:00|   1|        null|                null|            213.36|                    2|    EWR|Newark Airport|         EWR|
|2020-01-03 02:00:00|   1|      117.39|                   1|             100.1|                    1|    EWR|Newark Airport|         EWR|
|2020-01-03 03:00:00|   1|        